https://medium.com/@alishbakhalid058/virtual-makeup-lipstick-using-python-9abd7c0bd477

In [1]:
import cv2
import mediapipe as mp
import numpy as np

# Define lip landmarks for the upper and lower lips
upper_lip = [61, 185, 40, 39, 37, 0, 267, 269, 270, 408, 415, 272, 271, 268, 12, 38, 41, 42, 191, 78, 76]
lower_lip = [61, 146, 91, 181, 84, 17, 314, 405, 320, 307, 308, 324, 318, 402, 317, 14, 87, 178, 88, 95]

# Function to detect landmarks using MediaPipe
def detect_landmarks(src: np.ndarray) -> list | None:
    """
    Detect the landmarks of the face using MediaPipe face mesh.
    :param src: Input image (BGR format).
    :return: A list of landmarks or None if no landmarks are detected.
    """
    mp_face_mesh = mp.solutions.face_mesh
    face_mesh = mp_face_mesh.FaceMesh()  # Initialize the face mesh detector
    #rgb = cv2.cvtColor(src, cv2.COLOR_BGR2RGB)  # Convert BGR image to RGB
    results = face_mesh.process(src)  # Process the image to get landmarks

    if results.multi_face_landmarks:  # If landmarks are detected
        return results.multi_face_landmarks[0].landmark  # Return the first face's landmarks
    return None  # Return None if no landmarks are detected

# Function to extract and normalize landmarks for lips
def landmarks(landmarks: list, height: int, width: int, mask: list | None) -> np.ndarray:
    """
    Extract and normalize the lip landmarks for the image.
    :param landmarks: List of face landmarks.
    :param height: Height of the input image.
    :param width: Width of the input image.
    :param mask: A list of indices to extract the lip region.
    :return: Array of lip landmarks with normalized coordinates.
    """
    lip_landmarks = np.array([(int(landmark.x * width), int(landmark.y * height)) for landmark in landmarks])  # Normalize coordinates
    if mask:  # If a mask is provided
        lip_landmarks = lip_landmarks[mask]  # Apply the mask to filter specific landmarks
    return lip_landmarks

# Function to create a mask for lipstick application
def lip_mask(src: np.ndarray, points: np.ndarray, color: list) -> np.ndarray:
    """
    Create a mask for the lip area where the lipstick will be applied.
    :param src: Input image.
    :param points: Array of points representing the lip region.
    :param color: Color of the lipstick in BGR format.
    :return: A mask image with the lipstick applied to the lip region.
    """
    mask = np.zeros_like(src)  # Create a black mask (same size as the input image)
    mask = cv2.fillPoly(mask, [points], color)  # Fill the lip area with the chosen color
    mask = cv2.GaussianBlur(mask, (7, 7), 5)  # Apply Gaussian blur to smooth the edges of the mask
    return mask

# Function to apply lipstick to the image
def apply_lipstick(src: np.ndarray, color: str = "NO") -> np.ndarray:
    """
    Apply lipstick to the image by overlaying a color mask on the lips.
    :param src: Input image (BGR format).
    :param color: Color of the lipstick.
    :return: Image with lipstick applied.
    """
    ret_landmarks = detect_landmarks(src)  # Detect face landmarks in the image
    if ret_landmarks is None:  # If no landmarks are found, return the original image
        return src

    height, width, _ = src.shape  # Get image dimensions
    feature_landmarks = landmarks(ret_landmarks, height, width, upper_lip + lower_lip)  # Extract lip landmarks

    # Choose lipstick color based on input
    if color == 'orange':
        mask = lip_mask(src, feature_landmarks, [0, 143, 255])
    elif color == 'purple':
        mask = lip_mask(src, feature_landmarks, [255, 0, 0])
    elif color == 'NO':
        mask = lip_mask(src, feature_landmarks, [0, 0, 0])
    elif color == 'pink':
        mask = lip_mask(src, feature_landmarks, [153, 0, 157])
    elif color == 'green':
        mask = lip_mask(src, feature_landmarks, [0, 255, 0])
    elif color == 'berry':
        mask = lip_mask(src, feature_landmarks, [40, 0, 100])
    elif color == 'caramel':
        mask = lip_mask(src, feature_landmarks, [50, 70, 70])
    elif color == 'yellow':
        mask = lip_mask(src, feature_landmarks, [0, 255, 255])
    elif color == 'aqua':
        mask = lip_mask(src, feature_landmarks, [255, 255, 0])
    elif color == 'peach':
        mask = lip_mask(src, feature_landmarks, [35, 35, 139])
    elif color == 'red':
        mask = lip_mask(src, feature_landmarks, [2, 1, 159])

    # Combine the original image and the mask
    src1 = cv2.cvtColor(src, cv2.COLOR_BGR2RGB)  # Convert original image to RGB
    src1 = src
    mask1 = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)  # Convert mask to RGB
    mask1 = mask
    output = cv2.addWeighted(src1, 1.0, mask1, 0.4, 0.0)  # Blend the original and masked images
    return output

# Choose your settings here
video_source: int = 0  # 0 for webcam, or you can specify a video file path (e.g., "Videos/v1.mp4")
lip_color: str = 'red'  # Choose lip color: 'orange', 'purple', 'pink', 'green', etc.

# Initialize the video capture
cap = cv2.VideoCapture(video_source)

while True:
    ret, img = cap.read()  # Read a frame from the video capture
    if not ret:  # If reading the frame failed, break the loop
        break

    # Resize image for consistent display
    img = cv2.resize(img, (700, 400))  # Resize the frame to a fixed size (700x400)
    img = cv2.flip(img, 1)  # Flip the image horizontally for mirror effect

    # Apply lipstick
    output = apply_lipstick(img, lip_color)

    # Show the output
    cv2.imshow("Lipstick Application", output)

    # Exit if the user presses the 'Esc' key
    if cv2.waitKey(1) & 0xFF == 27:
        break

# Release the video capture and close the window
cap.release()  # Release the video capture object
cv2.destroyAllWindows()  # Close all OpenCV windows


In [6]:
import cv2
import mediapipe as mp
import numpy as np

class MakeupApplication:
    def __init__(self, lip_color: str = "NO", eyeliner_color: list = [0, 0, 0], eyeliner_thickness: int = 2, debug: bool = False):
        self.lip_color = lip_color
        self.eyeliner_color = eyeliner_color
        self.eyeliner_thickness = eyeliner_thickness
        self.debug = debug

        self.upper_lip = [61, 185, 40, 39, 37, 0, 267, 269, 270, 408, 415, 272, 271, 268, 12, 38, 41, 42, 191, 78, 76]
        self.lower_lip = [61, 146, 91, 181, 84, 17, 314, 405, 320, 307, 308, 324, 318, 402, 317, 14, 87, 178, 88, 95]
        self.left_eye = [33, 246, 161, 160, 159, 158, 157, 173]
        self.right_eye = [263, 466, 388, 387, 386, 385, 384, 398]

        self.mp_face_mesh = mp.solutions.face_mesh

    def detect_landmarks(self, src: np.ndarray) -> list | None:
        with self.mp_face_mesh.FaceMesh() as face_mesh:
            results = face_mesh.process(cv2.cvtColor(src, cv2.COLOR_BGR2RGB))
            if results.multi_face_landmarks:
                return results.multi_face_landmarks[0].landmark
        return None

    def extract_landmarks(self, landmarks: list, height: int, width: int, mask: list) -> np.ndarray:
        return np.array([(int(landmark.x * width), int(landmark.y * height)) for landmark in landmarks])[mask]

    def create_mask(self, src: np.ndarray, points: np.ndarray, color: list, thickness: int = -1) -> np.ndarray:
        mask = np.zeros_like(src)
        if thickness == -1:
            mask = cv2.fillPoly(mask, [points], color)
        else:
            mask = cv2.polylines(mask, [points], isClosed=False, color=color, thickness=thickness)
        return cv2.GaussianBlur(mask, (7, 7), 5)

    def apply_makeup(self, src: np.ndarray) -> np.ndarray:
        ret_landmarks = self.detect_landmarks(src)
        if ret_landmarks is None:
            return src

        height, width, _ = src.shape

        # Extract landmarks for lips and eyes
        lip_landmarks = self.extract_landmarks(ret_landmarks, height, width, self.upper_lip + self.lower_lip)
        left_eye_landmarks = self.extract_landmarks(ret_landmarks, height, width, self.left_eye)
        right_eye_landmarks = self.extract_landmarks(ret_landmarks, height, width, self.right_eye)

        # Create an overlay to blend the masks
        overlay = src.copy()

        # Apply lipstick
        if self.lip_color != "NO":
            lip_color_map = {
                'orange': [0, 143, 255],
                'purple': [255, 0, 0],
                'pink': [153, 0, 157],
                'green': [0, 255, 0],
                'berry': [40, 0, 100],
                'caramel': [50, 70, 70],
                'yellow': [0, 255, 255],
                'aqua': [255, 255, 0],
                'peach': [35, 35, 139],
                'red': [2, 1, 159]
            }
            lip_mask = self.create_mask(src, lip_landmarks, lip_color_map.get(self.lip_color, [0, 0, 0]))
            overlay = cv2.addWeighted(overlay, 1.0, lip_mask, 0.4, 0.0)

        # Apply eyeliner
        mask_left_eye = self.create_mask(src, left_eye_landmarks, self.eyeliner_color, thickness=self.eyeliner_thickness)
        mask_right_eye = self.create_mask(src, right_eye_landmarks, self.eyeliner_color, thickness=self.eyeliner_thickness)
        overlay = cv2.addWeighted(overlay, 1.0, mask_left_eye, 0.6, 0.0)
        overlay = cv2.addWeighted(overlay, 1.0, mask_right_eye, 0.6, 0.0)

        # Debug Mode: Draw facial landmarks
        if self.debug:
            for landmark in ret_landmarks:
                x, y = int(landmark.x * width), int(landmark.y * height)
                cv2.circle(overlay, (x, y), 2, (0, 0, 255), -1)  # Red dots for debug

        return overlay


# Main script for video processing
if __name__ == "__main__":
    # Define user settings
    video_source = 0  # 0 for webcam or path to video file
    lip_color = 'red'  # Choose lipstick color: 'orange', 'purple', etc.
    eyeliner_color = [0, 0, 0]  # Black eyeliner
    debug_mode = True  # Enable debug mode to see landmarks

    # Initialize MakeupApplication instance
    makeup_app = MakeupApplication(lip_color=lip_color, eyeliner_color=eyeliner_color, debug=debug_mode)

    # Start video capture
    cap = cv2.VideoCapture(video_source)

    while True:
        ret, img = cap.read()
        if not ret:
            break

        img = cv2.resize(img, (700, 400))
        img = cv2.flip(img, 1)

        # Apply makeup
        output = makeup_app.apply_makeup(img)

        # Display the output
        cv2.imshow("Makeup Application", output)

        # Break the loop if 'Esc' key is pressed
        if cv2.waitKey(1) & 0xFF == 27:
            break

    cap.release()
    cv2.destroyAllWindows()
